In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

import os
import matplotlib.pyplot as plt

import sklearn
from sklearn.svm import SVC

from sklearn.metrics import balanced_accuracy_score, roc_auc_score, accuracy_score
from scipy.stats import zscore, pearsonr,uniform
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, StratifiedKFold,RandomizedSearchCV

from scipy.io import loadmat
from scipy.stats import ttest_ind

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
train_cat=pd.read_excel("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx")
#train_cat.head()

In [ ]:
train_cat.columns

In [ ]:
train_cat.info()

In [3]:
train_FCM=pd.read_csv("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TRAIN//TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv")
#train_FCM.head()

In [ ]:
train_FCM.columns

In [4]:
train_Quant=pd.read_excel("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx")
#train_Quant.head()

In [ ]:
train_Quant.columns

In [5]:
train_Solutions=pd.read_excel("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TRAIN/TRAINING_SOLUTIONS.xlsx")
#train_Solutions.head()

In [ ]:
train_Solutions.columns

In [21]:
train_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   participant_id                    1213 non-null   object 
 1   Basic_Demos_Enroll_Year           1213 non-null   int64  
 2   Basic_Demos_Study_Site            1213 non-null   int64  
 3   PreInt_Demos_Fam_Child_Ethnicity  1202 non-null   float64
 4   PreInt_Demos_Fam_Child_Race       1213 non-null   int64  
 5   MRI_Track_Scan_Location           1213 non-null   int64  
 6   Barratt_Barratt_P1_Edu            1213 non-null   int64  
 7   Barratt_Barratt_P1_Occ            1213 non-null   int64  
 8   Barratt_Barratt_P2_Edu            1213 non-null   int64  
 9   Barratt_Barratt_P2_Occ            1213 non-null   int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 94.9+ KB


In [4]:
for col in train_cat.select_dtypes(include='int').columns:
    train_cat[col]=train_cat[col].astype('category')

In [5]:
    # Creating a list of all of the columns except the first participant_id
    columns_to_encode=train_cat.columns[1:].tolist()
    # Print the columns to encode
    print("Columns to encode;", columns_to_encode)

Columns to encode; ['Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site', 'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ']


In [7]:
data_encoded=pd.get_dummies(train_cat[columns_to_encode],drop_first=True)
data_encoded=data_encoded.map(lambda x:1 if x is True else (0 if x is False else x))

In [9]:
data_encoded.head()

,PreInt_Demos_Fam_Child_Ethnicity,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,Basic_Demos_Enroll_Year_2018,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,PreInt_Demos_Fam_Child_Race_1,...,Barratt_Barratt_P2_Edu_21,Barratt_Barratt_P2_Occ_5,Barratt_Barratt_P2_Occ_10,Barratt_Barratt_P2_Occ_15,Barratt_Barratt_P2_Occ_20,Barratt_Barratt_P2_Occ_25,Barratt_Barratt_P2_Occ_30,Barratt_Barratt_P2_Occ_35,Barratt_Barratt_P2_Occ_40,Barratt_Barratt_P2_Occ_45
0,0.0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1.0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#Combine encoded columns with the rest of the DataFrame
cat_train_final=pd.concat([train_cat.drop(columns=columns_to_encode), data_encoded], axis=1)
#cat_train_final.head()

In [11]:
train_cat_FCM=pd.merge(cat_train_final, train_FCM, on='participant_id')
#train_cat_FCM.head()

In [12]:
train_df=pd.merge(train_cat_FCM, train_Quant, on='participant_id')
train_df.head()

,participant_id,PreInt_Demos_Fam_Child_Ethnicity,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,Basic_Demos_Enroll_Year_2018,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,UmrK0vMLopoR,0.0,1,0,0,0,0,0,0,0,...,0,6,1,5,0,5,1,0,10,NaN
1,CPaeQkhcjg7d,1.0,0,0,0,1,0,0,1,0,...,0,18,6,8,7,8,10,4,5,NaN
2,Nb4EetVPm3gs,1.0,1,0,0,0,0,0,0,0,...,1,14,2,8,5,7,6,4,9,8.239904
3,p4vPhVu91o4b,0.0,0,0,1,0,0,0,1,0,...,6,24,4,16,9,10,8,4,6,NaN
4,M09PXs7arQ5E,0.0,0,0,0,1,0,0,1,0,...,1,18,4,11,4,10,7,3,9,8.940679


In [ ]:
# Merging TEST 

In [13]:
test_cat=pd.read_excel("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx")
test_cat.head()

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,Cfwaf5FX7jWK,2022,4,0.0,0.0,4,21.0,30.0,18.0,30.0
1,vhGrzmvA3Hjq,2023,4,0.0,0.0,4,21.0,45.0,NaN,30.0
2,ULliyEXjy4OV,2022,4,0.0,0.0,4,21.0,40.0,18.0,40.0
3,LZfeAb1xMtql,2022,4,0.0,0.0,3,21.0,45.0,21.0,45.0
4,EnFOUv0YK1RG,2022,4,2.0,0.0,4,18.0,0.0,21.0,45.0


In [14]:
test_FCM=pd.read_csv("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TEST//TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv")
#test_FCM.head()

In [15]:
test_Quant=pd.read_excel("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx")
#test_Quant.head()

In [16]:
test_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   participant_id                    304 non-null    object 
 1   Basic_Demos_Enroll_Year           304 non-null    int64  
 2   Basic_Demos_Study_Site            304 non-null    int64  
 3   PreInt_Demos_Fam_Child_Ethnicity  301 non-null    float64
 4   PreInt_Demos_Fam_Child_Race       298 non-null    float64
 5   MRI_Track_Scan_Location           304 non-null    int64  
 6   Barratt_Barratt_P1_Edu            303 non-null    float64
 7   Barratt_Barratt_P1_Occ            303 non-null    float64
 8   Barratt_Barratt_P2_Edu            268 non-null    float64
 9   Barratt_Barratt_P2_Occ            262 non-null    float64
dtypes: float64(6), int64(3), object(1)
memory usage: 23.9+ KB


In [ ]:
# NaN values

In [25]:
test_cat.fillna({'PreInt_Demos_Fam_Child_Ethnicity':test_cat['PreInt_Demos_Fam_Child_Ethnicity'].mean()},inplace=True)
test_cat.fillna({'PreInt_Demos_Fam_Child_Race':test_cat['PreInt_Demos_Fam_Child_Race'].mean()},inplace=True)
test_cat.fillna({'Barratt_Barratt_P1_Edu':test_cat['Barratt_Barratt_P1_Edu'].mean()},inplace=True)
test_cat.fillna({'Barratt_Barratt_P1_Occ':test_cat['Barratt_Barratt_P1_Occ'].mean()},inplace=True)
test_cat.fillna({'Barratt_Barratt_P2_Edu':test_cat['Barratt_Barratt_P2_Edu'].mean()},inplace=True)
test_cat.fillna({'Barratt_Barratt_P2_Occ':test_cat['Barratt_Barratt_P2_Occ'].mean()},inplace=True)

In [26]:
print(test_cat.isna().sum())

participant_id                      0
Basic_Demos_Enroll_Year             0
Basic_Demos_Study_Site              0
PreInt_Demos_Fam_Child_Ethnicity    0
PreInt_Demos_Fam_Child_Race         0
MRI_Track_Scan_Location             0
Barratt_Barratt_P1_Edu              0
Barratt_Barratt_P1_Occ              0
Barratt_Barratt_P2_Edu              0
Barratt_Barratt_P2_Occ              0
dtype: int64


In [29]:
test_cat['PreInt_Demos_Fam_Child_Race']=test_cat['PreInt_Demos_Fam_Child_Race'].astype('category')
test_cat['MRI_Track_Scan_Location']=test_cat['MRI_Track_Scan_Location'].astype('category')
test_cat['Barratt_Barratt_P1_Edu']=test_cat['Barratt_Barratt_P1_Edu'].astype('category')
test_cat['Barratt_Barratt_P1_Occ']=test_cat['Barratt_Barratt_P1_Occ'].astype('category')
test_cat['Barratt_Barratt_P2_Edu']=test_cat['Barratt_Barratt_P2_Edu'].astype('category')
test_cat['Barratt_Barratt_P2_Occ']=test_cat['Barratt_Barratt_P2_Occ'].astype('category')

In [30]:
for col in test_cat.select_dtypes(include='int').columns:
    test_cat[col]=test_cat[col].astype('category')

In [31]:
test_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   participant_id                    304 non-null    object  
 1   Basic_Demos_Enroll_Year           304 non-null    category
 2   Basic_Demos_Study_Site            304 non-null    category
 3   PreInt_Demos_Fam_Child_Ethnicity  304 non-null    float64 
 4   PreInt_Demos_Fam_Child_Race       304 non-null    category
 5   MRI_Track_Scan_Location           304 non-null    category
 6   Barratt_Barratt_P1_Edu            304 non-null    category
 7   Barratt_Barratt_P1_Occ            304 non-null    category
 8   Barratt_Barratt_P2_Edu            304 non-null    category
 9   Barratt_Barratt_P2_Occ            304 non-null    category
dtypes: category(8), float64(1), object(1)
memory usage: 9.5+ KB


In [32]:
testcolumns_to_encode=test_cat.columns[1:].tolist()
print("Columns to encode:", testcolumns_to_encode)

Columns to encode: ['Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site', 'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ']


In [33]:
testdata_encoded=pd.get_dummies(test_cat[testcolumns_to_encode],drop_first=True)
testdata_encoded=testdata_encoded.map(lambda x:1 if x is True else (0 if x is False else x))

In [34]:
testdata_encoded

,PreInt_Demos_Fam_Child_Ethnicity,Basic_Demos_Enroll_Year_2020,Basic_Demos_Enroll_Year_2021,Basic_Demos_Enroll_Year_2022,Basic_Demos_Enroll_Year_2023,Basic_Demos_Study_Site_5,PreInt_Demos_Fam_Child_Race_1.0,PreInt_Demos_Fam_Child_Race_2.0,PreInt_Demos_Fam_Child_Race_2.2315436241610738,PreInt_Demos_Fam_Child_Race_3.0,...,Barratt_Barratt_P2_Occ_5.0,Barratt_Barratt_P2_Occ_10.0,Barratt_Barratt_P2_Occ_15.0,Barratt_Barratt_P2_Occ_20.0,Barratt_Barratt_P2_Occ_25.0,Barratt_Barratt_P2_Occ_30.0,Barratt_Barratt_P2_Occ_35.0,Barratt_Barratt_P2_Occ_35.76335877862596,Barratt_Barratt_P2_Occ_40.0,Barratt_Barratt_P2_Occ_45.0
0,0.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,0.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
300,0.0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
301,0.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
302,0.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [35]:
cat_test_final=pd.concat([test_cat.drop(columns=columns_to_encode), data_encoded], axis=1)
cat_test_final.head()

,participant_id,PreInt_Demos_Fam_Child_Ethnicity,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,Basic_Demos_Enroll_Year_2018,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,...,Barratt_Barratt_P2_Edu_21,Barratt_Barratt_P2_Occ_5,Barratt_Barratt_P2_Occ_10,Barratt_Barratt_P2_Occ_15,Barratt_Barratt_P2_Occ_20,Barratt_Barratt_P2_Occ_25,Barratt_Barratt_P2_Occ_30,Barratt_Barratt_P2_Occ_35,Barratt_Barratt_P2_Occ_40,Barratt_Barratt_P2_Occ_45
0,Cfwaf5FX7jWK,0.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,vhGrzmvA3Hjq,1.0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,ULliyEXjy4OV,1.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,LZfeAb1xMtql,0.0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,EnFOUv0YK1RG,0.0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
test_cat_FCM=pd.merge(cat_test_final, test_FCM, on='participant_id')
#test_cat_FCM.head()

In [37]:
test_df=pd.merge(test_cat_FCM, test_Quant, on='participant_id')
test_df.head()

,participant_id,PreInt_Demos_Fam_Child_Ethnicity,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,Basic_Demos_Enroll_Year_2018,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,Cfwaf5FX7jWK,0.0,1,0,0,0,0,0,0,0,...,2.0,12.0,3.0,9.0,2.0,7.0,3.0,0.0,8.0,8.992813
1,vhGrzmvA3Hjq,1.0,0,0,0,1,0,0,1,0,...,2.0,16.0,8.0,5.0,7.0,3.0,11.0,3.0,9.0,12.324093
2,ULliyEXjy4OV,1.0,1,0,0,0,0,0,0,0,...,1.0,7.0,1.0,6.0,1.0,5.0,1.0,0.0,9.0,7.770933
3,LZfeAb1xMtql,0.0,0,0,1,0,0,0,1,0,...,4.0,15.0,4.0,10.0,8.0,6.0,5.0,1.0,6.0,9.304814
4,EnFOUv0YK1RG,0.0,0,0,0,1,0,0,1,0,...,2.0,18.0,6.0,12.0,5.0,10.0,6.0,0.0,10.0,8.261350


In [ ]:
# NaN values

In [38]:
print(train_df.isna().sum())

participant_id                        0
PreInt_Demos_Fam_Child_Ethnicity     11
Basic_Demos_Enroll_Year_2016          0
Basic_Demos_Enroll_Year_2017          0
Basic_Demos_Enroll_Year_2018          0
                                   ... 
SDQ_SDQ_Hyperactivity                 0
SDQ_SDQ_Internalizing                 0
SDQ_SDQ_Peer_Problems                 0
SDQ_SDQ_Prosocial                     0
MRI_Track_Age_at_Scan               360
Length: 19973, dtype: int64


In [43]:
train_df.fillna({'MRI_Track_Age_at_Scan':train_df['MRI_Track_Age_at_Scan'].mean()},inplace=True)
train_df.fillna({'PreInt_Demos_Fam_Child_Ethnicity':train_df['PreInt_Demos_Fam_Child_Ethnicity'].mean()},inplace=True)

In [40]:
print(train_df.isna().sum())

participant_id                      0
PreInt_Demos_Fam_Child_Ethnicity    0
Basic_Demos_Enroll_Year_2016        0
Basic_Demos_Enroll_Year_2017        0
Basic_Demos_Enroll_Year_2018        0
                                   ..
SDQ_SDQ_Hyperactivity               0
SDQ_SDQ_Internalizing               0
SDQ_SDQ_Peer_Problems               0
SDQ_SDQ_Prosocial                   0
MRI_Track_Age_at_Scan               0
Length: 19973, dtype: int64


In [44]:
print(test_df.isna().sum())

participant_id                       0
PreInt_Demos_Fam_Child_Ethnicity     0
Basic_Demos_Enroll_Year_2016         0
Basic_Demos_Enroll_Year_2017         0
Basic_Demos_Enroll_Year_2018         0
                                    ..
SDQ_SDQ_Hyperactivity               30
SDQ_SDQ_Internalizing               30
SDQ_SDQ_Peer_Problems               30
SDQ_SDQ_Prosocial                   30
MRI_Track_Age_at_Scan                0
Length: 19973, dtype: int64


In [45]:
#test_df.fillna({'PreInt_Demos_Fam_Child_Ethnicity':test_df['PreInt_Demos_Fam_Child_Ethnicity'].mean()},inplace=True)
#test_df.fillna({'PreInt_Demos_Fam_Child_Race':test_df['PreInt_Demos_Fam_Child_Race'].mean()},inplace=True)
#test_df.fillna({'Barratt_Barratt_P1_Edu':test_df['Barratt_Barratt_P1_Edu'].mean()},inplace=True)
#test_df.fillna({'Barratt_Barratt_P1_Occ':test_df['Barratt_Barratt_P1_Occ'].mean()},inplace=True)
test_df.fillna({'SDQ_SDQ_Hyperactivity':test_df['SDQ_SDQ_Hyperactivity'].mean()},inplace=True)
test_df.fillna({'SDQ_SDQ_Internalizing':test_df['SDQ_SDQ_Internalizing'].mean()},inplace=True)
test_df.fillna({'SDQ_SDQ_Peer_Problems':test_df['SDQ_SDQ_Peer_Problems'].mean()},inplace=True)
test_df.fillna({'SDQ_SDQ_Prosocial':test_df['SDQ_SDQ_Prosocial'].mean()},inplace=True)

In [46]:
print(test_df.isna().sum())

participant_id                      0
PreInt_Demos_Fam_Child_Ethnicity    0
Basic_Demos_Enroll_Year_2016        0
Basic_Demos_Enroll_Year_2017        0
Basic_Demos_Enroll_Year_2018        0
                                   ..
SDQ_SDQ_Hyperactivity               0
SDQ_SDQ_Internalizing               0
SDQ_SDQ_Peer_Problems               0
SDQ_SDQ_Prosocial                   0
MRI_Track_Age_at_Scan               0
Length: 19973, dtype: int64


In [ ]:
# X_train Y_train

In [47]:
X_train=train_df.drop(columns=['participant_id'])
Y_train=train_Solutions.drop(columns=['participant_id'])

In [48]:
# proveriti da li je instaliran xgboost?
# u Anaconda Prompt ukucaj: conda list xgboost
# ako nij instaliran nece ga biti na listi
#INSTALACIJA: u Anaconda Prompt ukucaj: conda install -c conda-forge xgboost
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
#initialize the base classifier
xgb_classifier=XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.1, max_depth=5)

In [49]:
#Wrap with MultiOutputClassifier for multi-target classification
multioutput_classifier=MultiOutputClassifier(xgb_classifier)

In [50]:
#Train the model
# It takes about 10 minutes to run
multioutput_classifier.fit(X_train, Y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=0.1, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=5,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...))

In [ ]:
#Testing the model

In [51]:
participant_id=test_df['participant_id']
X_test=test_df.drop(columns='participant_id')


In [ ]:
train_df.columns

In [ ]:
test_df.columns

In [52]:
y_pred=multioutput_classifier.predict(X_test)

In [54]:
# Convert predictions to a DataFrame
predictions_df=pd.DataFrame(y_pred, columns=['Predicted_Gender','Predicted_ADHD'])

# Combine participant IDs with predictions
result_df=pd.concat([participant_id.reset_index(drop=True), predictions_df], axis=1)

# Print or save the DataFrame
print(result_df)

    participant_id  Predicted_Gender  Predicted_ADHD
0     Cfwaf5FX7jWK                 0               0
1     vhGrzmvA3Hjq                 1               0
2     ULliyEXjy4OV                 1               0
3     LZfeAb1xMtql                 0               0
4     EnFOUv0YK1RG                 1               0
..             ...               ...             ...
299   UadZfjdEg7eG                 0               0
300   IUEHiLmQAqCi                 1               1
301   cRySmCadYFRO                 0               0
302   E3MvDUtJadc5                 0               0
303   dQJXfyRazknD                 1               0

[304 rows x 3 columns]


In [ ]:
# Cross Validation

In [56]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score

In [57]:
def multi_output_accuracy(y_true, y_pred):
    #Ensure y_true and y_pred are NumPy arrays
    y_true=np.array(y_true)
    y_pred=np.array(y_pred)
    # Compare accuracy for each target variable and return the mean
    return np.mean([accuracy_score(y_true[:,i], y_pred[:,i]) for i in range(y_true.shape[1])])

In [58]:
# Create a scorer using scikit-learn's make_scorer
multi_output_scorer=make_scorer(multi_output_accuracy)

In [59]:
# Note; It takes 12 min to run this cell
# Perform cross-validation on the training data
cv_scores=cross_val_score(multioutput_classifier, X_train, Y_train, cv=5, scoring=multi_output_scorer)

# Output the cross-validation results
print("Cross-validation scores for each fold:", cv_scores)
print("Mean CV score:", np.mean(cv_scores))

Cross-validation scores for each fold: [0.82304527 0.78600823 0.69341564 0.64669421 0.33471074]
Mean CV score: 0.6567748188960311
